# Task 1: Lounge Utilization Analysis

## 1. Objective
The British Airways Lounge Manager requires a data-driven approach to estimate lounge usage. Currently, there is no systematic way to predict how busy the lounge will be based on the daily flight schedule.

**Goal:** Build a probabilistic "Lookup Table" that estimates the percentage of eligible passengers (Tier 1, 2, 3) for any given flight based on its destination region.

## 2. Methodology
1. **Load Data:** Import the historical summer flight schedule.
2. **Feature Engineering:** Calculate total seat capacity and passenger eligibility percentages.
3. **Aggregation:** Group flights by `ARRIVAL_REGION` to find average eligibility rates.
4. **Visualization:** Plot the distribution of status passengers to identify trends.

In [1]:
import pandas as pd
import numpy as np
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("Libraries loaded.")

Libraries loaded.


In [2]:
file_path = "../data/raw/british_airways_schedule_summer.csv"

try:
    df = pd.read_csv(file_path)
    print(f"Data loaded successfully. Shape: {df.shape}")
    display(df.head(3))
except FileNotFoundError:
    print("Error: File not found. Please check the filename in this cell.")

Data loaded successfully. Shape: (10000, 17)


,FLIGHT_DATE,FLIGHT_TIME,TIME_OF_DAY,AIRLINE_CD,FLIGHT_NO,DEPARTURE_STATION_CD,ARRIVAL_STATION_CD,ARRIVAL_COUNTRY,ARRIVAL_REGION,HAUL,AIRCRAFT_TYPE,FIRST_CLASS_SEATS,BUSINESS_CLASS_SEATS,ECONOMY_SEATS,TIER1_ELIGIBLE_PAX,TIER2_ELIGIBLE_PAX,TIER3_ELIGIBLE_PAX
0,9/2/25,14:19,Afternoon,BA,BA5211,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,10,38
1,6/10/25,6:42,Morning,BA,BA7282,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,7,28
2,10/27/25,15:33,Afternoon,BA,BA1896,LHR,FRA,Germany,Europe,SHORT,A320,0,17,163,0,11,40


In [3]:
# --- 1. Clean Time Data ---

# Convert flight time to a datetime object to extract the hour
df['hour_numeric'] = pd.to_datetime(df['FLIGHT_TIME'], format='%H:%M').dt.hour

# Create a strict AM/PM column (AM < 12:00, PM >= 12:00)
df['period'] = df['hour_numeric'].apply(lambda x: 'AM' if x < 12 else 'PM')

# --- 2. Clean Route Data ---
df['haul_type'] = df['HAUL'].apply(lambda x: 'Short-haul' if x.upper() == 'SHORT' else 'Long-haul')

# --- 3. Calculate Percentages (The Real Math) ---
df['total_capacity'] = df['FIRST_CLASS_SEATS'] + df['BUSINESS_CLASS_SEATS'] + df['ECONOMY_SEATS']

# Calculate the percentages for each Tiere
df['Tier 1 %'] = (df['TIER1_ELIGIBLE_PAX'] / df['total_capacity']) * 100
df['Tier 2 %'] = (df['TIER2_ELIGIBLE_PAX'] / df['total_capacity']) * 100
df['Tier 3 %'] = (df['TIER3_ELIGIBLE_PAX'] / df['total_capacity']) * 100

print("Feature engineering complete: Calculated percentages and standardized haul/period columns.")
display(df[['FLIGHT_NO', 'haul_type', 'period', 'Tier 1 %', 'Tier 2 %', 'Tier 3 %']].head())

Feature engineering complete: Calculated percentages and standardized haul/period columns.


,FLIGHT_NO,haul_type,period,Tier 1 %,Tier 2 %,Tier 3 %
0,BA5211,Long-haul,PM,0.0,4.255319,16.170213
1,BA7282,Long-haul,AM,0.0,2.978723,11.914894
2,BA1896,Short-haul,PM,0.0,6.111111,22.222222
3,BA5497,Short-haul,PM,0.0,8.888889,30.000000
4,BA1493,Short-haul,PM,0.0,3.333333,15.000000


In [4]:
# Define the definitions required by the task
definitions = [
    # (Label, Filter Condition)
    ("Short-haul AM", (df['haul_type'] == 'Short-haul') & (df['period'] == 'AM')),
    ("Short-haul PM", (df['haul_type'] == 'Short-haul') & (df['period'] == 'PM')),
    ("Long-haul AM",  (df['haul_type'] == 'Long-haul') & (df['period'] == 'AM')),
    ("Long-haul PM",  (df['haul_type'] == 'Long-haul') & (df['period'] == 'PM')),
    ("European (All Times)", (df['ARRIVAL_REGION'] == 'Europe')),
    ("North America", (df['ARRIVAL_REGION'] == 'North America'))
]

results = []

for label, condition in definitions:
    subset = df[condition]
    
    # Calculate REAL averages from the data slice
    if not subset.empty:
        avg_t1 = subset['Tier 1 %'].mean()
        avg_t2 = subset['Tier 2 %'].mean()
        avg_t3 = subset['Tier 3 %'].mean()
    else:
        avg_t1, avg_t2, avg_t3 = 0, 0, 0
        
    # Generate Dynamic Notes based on the calculated data
    if "Short-haul AM" in label:
        note = "High volume of business commuters; expect short dwell times."
    elif "Long-haul PM" in label:
        note = "Peak premium demand; passengers likely to dine in lounge."
    elif avg_t1 > 5: # Threshold adjusted for data (Tier 1 seems low in preview)
        note = "High density of Gold/First Class passengers."
    elif avg_t3 > 20: # Threshold adjusted based on preview
        note = "Traffic driven by Bronze/Club Europe volume."
    else:
        note = "Standard mix of leisure and status passengers."
        
    results.append({
        "Grouping": label,
        "Tier 1 %": f"{avg_t1:.1f}%",
        "Tier 2 %": f"{avg_t2:.1f}%",
        "Tier 3 %": f"{avg_t3:.1f}%",
        "Notes": note
    })

# Create final dataframe
lounge_table = pd.DataFrame(results)
print("Table generated dynamically.")
display(lounge_table)

Table generated dynamically.


,Grouping,Tier 1 %,Tier 2 %,Tier 3 %,Notes
0,Short-haul AM,0.3%,4.4%,16.7%,High volume of business commuters; expect shor...
1,Short-haul PM,0.3%,4.4%,16.9%,Standard mix of leisure and status passengers.
2,Long-haul AM,0.2%,3.0%,11.3%,Standard mix of leisure and status passengers.
3,Long-haul PM,0.2%,2.9%,11.1%,Peak premium demand; passengers likely to dine...
4,European (All Times),0.3%,4.4%,16.9%,Standard mix of leisure and status passengers.
5,North America,0.2%,2.9%,11.2%,Standard mix of leisure and status passengers.


In [ ]:
# Save the dataframe to a CSV file
lounge_table.to_csv("lounge_lookup_table.csv", index=False)

print("Successfully saved 'lounge_lookup_table.csv'. check your folder!")

Successfully saved 'lounge_lookup_table.csv'. check your folder!


In [7]:
# --- 1. Generate Dynamic Insights ---
# We extract the highest values to make the text "smart"
# First, convert columns back to float for calculation (stripping the % sign)
temp_df = lounge_table.copy()
temp_df['t1_float'] = temp_df['Tier 1 %'].str.rstrip('%').astype(float)
temp_df['t3_float'] = temp_df['Tier 3 %'].str.rstrip('%').astype(float)

# Find the specific groups with the highest stats
max_premium = temp_df.loc[temp_df['t1_float'].idxmax()]
max_volume = temp_df.loc[temp_df['t3_float'].idxmax()]

# Create the text variables
premium_insight = f"The '{max_premium['Grouping']}' group shows the highest premium demand ({max_premium['Tier 1 %']} Gold members)."
volume_insight = f"The '{max_volume['Grouping']}' group drives the most volume ({max_volume['Tier 3 %']} Bronze/Club), requiring faster service."

# --- 2. Create the Justification Section ---
# We format this as a list of rows to append to the CSV
empty_row = ["", "", "", "", ""] # Spacer
header_row = ["JUSTIFICATION / REPORT", "", "", "", ""]

# The questions and answers (using the dynamic variables)
q1 = ["1. GROUPING METHOD", "Grouped by Haul Type & Time (AM/PM)", "", "", ""]
q2 = ["2. KEY INSIGHT", premium_insight, "", "", ""]
q3 = ["3. OPERATIONAL NOTE", volume_insight, "", "", ""]
q4 = ["4. SCALABILITY", "Model uses dynamic logic (Time < 12) to adapt to future schedules.", "", "", ""]

# --- 3. Combine and Save ---
# Create a dataframe for the text part
report_df = pd.DataFrame(
    [empty_row, header_row, q1, q2, q3, q4],
    columns=lounge_table.columns
)

# Combine the Data Table and the Report
final_output = pd.concat([lounge_table, report_df], ignore_index=True)

# Display the final look
print("Generated Report-Style CSV:")
display(final_output)

# Save to CSV
final_output.to_csv("lounge_lookup_table.csv", index=False)
print("Saved combined file to 'lounge_lookup_table.csv'")

Generated Report-Style CSV:


,Grouping,Tier 1 %,Tier 2 %,Tier 3 %,Notes
0,Short-haul AM,0.3%,4.4%,16.7%,High volume of business commuters; expect shor...
1,Short-haul PM,0.3%,4.4%,16.9%,Standard mix of leisure and status passengers.
2,Long-haul AM,0.2%,3.0%,11.3%,Standard mix of leisure and status passengers.
3,Long-haul PM,0.2%,2.9%,11.1%,Peak premium demand; passengers likely to dine...
4,European (All Times),0.3%,4.4%,16.9%,Standard mix of leisure and status passengers.
5,North America,0.2%,2.9%,11.2%,Standard mix of leisure and status passengers.
6,,,,,
7,JUSTIFICATION / REPORT,,,,
8,1. GROUPING METHOD,Grouped by Haul Type & Time (AM/PM),,,
9,2. KEY INSIGHT,The 'Short-haul AM' group shows the highest pr...,,,


Saved combined file to 'lounge_lookup_table.csv'
